In [1]:
import tensorflow as tf
import numpy as np
from tfUtils import *
import matplotlib.pyplot as plt
import innvestigate

Using TensorFlow backend.


In [2]:
def generate_data(m, n_x, seed = 9):
    np.random.seed = seed
    X = np.random.randn(n_x, m)
    Y = np.zeros((2, m))
    dist = np.random.randint(low = 0, high = n_x-1, size = m)
    for i in range(m):
        t = np.random.rand();
        if t < 0.5:
            X[dist[i], i] = -20
            Y[0, i] = 1
        elif t >= 0.5:
            X[dist[i], i] = 20
            Y[1, i] = 1
    return X, Y, dist

In [3]:
X_train, Y_train, dist = generate_data(40000, 100)

In [4]:
def create_placeholders(n_x, n_y):
    X = tf.placeholder(tf.float32, [n_x, None])
    Y = tf.placeholder(tf.float32, [n_y, None])
    return X, Y

In [5]:
def initialize_parameters(n_x, layers):
    parameters = {}
    layers.insert(0, n_x)
    for i in range(1, len(layers)):
        parameters["W" + str(i)] = tf.get_variable("W" + str(i), [layers[i], layers[i-1]],
                                                  initializer = tf.contrib.layers.xavier_initializer())
        parameters["b" + str(i)] = tf.get_variable("b" + str(i), [layers[i], 1], 
                                                  initializer = tf.zeros_initializer())
    return parameters

In [6]:
def forward_propagation(A, W, b, activation = "relu"):
    Z = tf.matmul(W, A) + b
    if activation == "relu":
        A = tf.nn.relu(Z)
    elif activation == "sigmoid":
        A = tf.nn.sigmoid(Z)
    elif activation == "tanh":
        A = tf.nn.tanh(Z)
    elif activation == "linear":
        A = Z
    return A

In [7]:
def compute_cost(Z, Y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = tf.transpose(Z), labels = tf.transpose(Y)))
    return cost

In [82]:
def model(X_train, Y_train, layers, activations, epochs, batch_size, starting_rate, decay, lambd):
    tf.reset_default_graph()
    n_x, m = X_train.shape
    n_y, __ = Y_train.shape
    X, Y = create_placeholders(n_x, n_y)
    l = len(layers)
    parameter_list = []
    parameters = initialize_parameters(n_x, layers)
    A = X
    for i in range(1, len(layers)):
        A = forward_propagation(A, parameters["W" + str(i)], parameters["b" + str(i)], activations[i - 1])
    cost = compute_cost(A, Y)
    hard_A = tf.argmax(tf.nn.softmax(A, dim = 0), axis = 0)
    acc = tf.reduce_mean(hard_A * tf.argmax(Y, axis = 0))
    for i in range(1, len(layers)):
        cost += lambd * tf.nn.l2_loss(parameters["W" + str(i)])
    global_steps = tf.Variable(0, trainable = False)
    learning_rate = tf.train.exponential_decay(starting_rate, global_steps, 5000, decay, staircase = True)
    train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost, global_step = global_steps)
    init = tf.global_variables_initializer()
    cost_list = []
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(epochs):
            minibatches = random_minibatches(X_train, Y_train, batch_size, seed = epoch)
            epoch_cost = 0
            for minibatch in minibatches:
                sess.run(train, feed_dict = {X : minibatch[0], Y : minibatch[1]})
                epoch_cost += sess.run(cost, feed_dict = {X : minibatch[0], Y : minibatch[1]}) / len(minibatches)
            cost_list.append(epoch_cost)        
            if epoch % 5 == 0:
                print(epoch_cost)
        parameter_names = ["W1", "b1", "W2", "b2", "W3", "b3", "W4", "b4"]
        for j in range(len(parameter_names)):
            parameter_list.append(sess.run(parameters[parameter_names[j]]))
        print("accuracy", sess.run(acc, feed_dict = {X : X_train, Y : Y_train}))
    sess.close()
    plt.plot(np.array(cost_list), '-b')
    return parameter_list

In [83]:
tf.reset_default_graph()

In [84]:
layers = [200, 300, 100, 2]
activations = ["relu", "relu", "tanh", "linear"]
epochs = 30
batch_size = 32
starting_rate = 0.0003
decay = .9
lambd = 0

In [85]:
parameter_list = model(X_train, Y_train, layers , activations, epochs, batch_size, starting_rate, decay, lambd)

Instructions for updating:
dim is deprecated, use axis instead
0.05418745292760415
1.636515573954966e-05
5.430723538861304e-07
1.9514555957478364e-08


KeyboardInterrupt: 

In [ ]:
from keras.layers import Input, Activation, Dense, Flatten
from keras.models import Model

In [ ]:
X_train_ = X_train.reshape(-1, 10, 10, 1)
Y_train_ = Y_train.reshape(-1, 2)

In [ ]:
def create_model(input_shape, layers, activations):
    X_input = Input(input_shape)
    X = Flatten()(X_input)
    X = Dense(layers[0], activation = activations[0])(X)
    X = Dense(layers[1], activation = activations[1])(X)
    X = Dense(layers[2], activation = activations[2])(X)
    X = Dense(layers[3], activation = activations[3])(X)
    model = Model(inputs = X_input, outputs = X)
    return model    

In [ ]:
layers = [200, 300, 100, 2]
activations = ["relu", "relu", "tanh", "linear"]

In [ ]:
model = create_model((10, 10, 1), layers, activations)

In [ ]:
for j in range(len(parameter_list)):
    if j % 2 == 0:
        parameter_list[j] = parameter_list[j].T
    else:
        parameter_list[j] = parameter_list[j].reshape(-1)

In [ ]:
model.set_weights(parameter_list)

In [ ]:
analyzer = innvestigate.create_analyzer("lrp.alpha_2_beta_1_IB", model)

In [ ]:
len(parameter_list)

In [ ]:
analysis = analyzer.analyze(X_train_[0:100,:])

In [ ]:
dist

In [ ]:
rel_me = np.zeros((len(dist), 100))
for i in range(len(dist)):
    rel_me[i, dist[i]] = 1

In [ ]:
rel_me = np.reshape(rel_me, (-1, 10, 10)) 

In [ ]:
plt.close()
for i in range(100):
    f, axarr = plt.subplots(1, 2, figsize=(10, 10))
    m = np.max(np.abs(analysis[i,:,:,0]))
    a = ((analysis[i,:,:,0]/m)+1)/2
    fig = axarr[0].imshow(a, vmax = 1, vmin = 0, cmap = "jet")
    fig = axarr[1].imshow(rel_me[i,:,:], cmap = "binary")
plt.show()

In [ ]:
for j in range(len(parameter_list)):
    if j % 2 == 0:
        parameter_list[j] = parameter_list[j].T
    else:
        parameter_list[j] = parameter_list[j].reshape(-1, 1)

In [ ]:
A1 = forward_propagation(X_train, parameter_list[0], parameter_list[1], 'relu')
A2 = forward_propagation(A1, parameter_list[2], parameter_list[3], 'relu')
A3 = forward_propagation(A2, parameter_list[4], parameter_list[5], 'tanh')
A4 = tf.nn.softmax(forward_propagation(A1, parameter_list[6], parameter_list[7], 'linear'))

In [67]:
a = np.array([[0, 1, 0], [1, 0, 0]]).T

In [68]:
a

array([[0, 1],
       [1, 0],
       [0, 0]])

In [69]:
np.argmax(a, axis = 0)

array([1, 0])